## Landmark Extraction Pipeline

Extract a point cloud-based representation of fluorescence intensity distributions from image stacks where the overall tissue has previously been masked using `RUN_preprocessing.ipynb`. Further to that, align the point clouds to the image frame and use the apical center point of the rosette (the lumen) as a reference point for registration, then normalize the z-axis to match the extent of the apico-basal axis across samples. Finally, compute the Euclidean distance of points in the point cloud to the lumen as an additional simplified measure.

This pipeline can be run both on the raw intensities and on the membrane-subtracted intensities, with the latter generally giving more insight into cytoplasmic/vesicle localization since in the former the membrane can overshadow everything else.

**Note:** For this to run the location of the apical center (the lumen position) of the neuromast has to be determined manually and its coordinates (in pixels) must be written in `<fpath>\metadata.xlsx`, which then has to be exported as a *tab-separated text file* called `<fpath>\metadata.txt`!

### Preparations

In [ ]:
# External imports
from __future__ import division
import os, sys
import numpy as np
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
from tifffile import imread
%matplotlib inline

# Internal imports
import quant.pointcloud as pcl

### Single Run

In [ ]:
# Specify parameters
fpath  = r'data_ex'
fname  = r'example_stack_wt_8bit_masked.tif'
res    = [0.22, 0.06, 0.06]

# Load data
stack = imread(os.path.join(fpath,fname))
stack = np.rollaxis(stack,1)  # Roll channels to the front
mem_stack = stack[0,...]
cx7_stack = stack[1,...]

# Run pipeline
print "Running with cxcr7b as channel and membranes as reference..."
lm_cx7, lm_cx7_tf, lum_dist_cx7 = pcl.point_cloud_pipeline(cx7_stack, mem_stack,
                                                           fpath, fname, res, num_LMs=1000,
                                                           verbose=True, show=True)
print "\n\nRunning with membranes as channel and also membranes as reference..."
lm_mem, lm_mem_tf, lum_dist_mem = pcl.point_cloud_pipeline(mem_stack, mem_stack,
                                                           fpath, fname, res, num_LMs=1000,
                                                           verbose=True, show=True)

### Batch Run

In [ ]:
# Specify parameters
fpath     = r'data_ex'
fname_end = r'_masked.tif'
res       = [0.22, 0.06, 0.06]
c_names   = ['mem', 'cx7']

# Get filenames
fnames = [fname for fname in os.listdir(fpath) if fname.endswith(fname_end)]

# Run pipeline and save results
for fname in fnames:
    
    # Report
    print "## Starting run on", fname
    
    # Import data
    stack = imread(os.path.join(fpath,fname))
    stack = np.rollaxis(stack,1)  # Roll channels to the front
    
    # Run pipeline
    output_dict = {}
    for c,n in zip(range(stack.shape[0]), c_names):
        lms, lms_tf, lum_dist_lms = pcl.point_cloud_pipeline(stack[c,...], stack[0,...],
                                                             fpath, fname, res, num_LMs=1000)
        output_dict['lm_'+n]       = lms
        output_dict['lm_'+n+'_tf'] = lms_tf
        output_dict['lum_dist_'+n] = lum_dist_lms
    
    # Save results
    np.savez(os.path.join(fpath, fname[:-4]+'_LMs'), **output_dict)

# report
print "### ALL DONE! ###"

### Batch Run for Membrane-Subtracted Data

In [ ]:
# Specify parameters
fpath     = r'data_ex'
fname_end = r'_memsub.tif'
res       = [0.22, 0.06, 0.06]
c_names   = ['mem', 'cx7']

# Get filenames
fnames = [fname for fname in os.listdir(fpath) if fname.endswith(fname_end)]

# Run pipeline and save results
for fname in fnames:
    
    # Report
    print "## Starting run on", fname
    
    # Import data
    stack = imread(os.path.join(fpath,fname))
    stack = np.rollaxis(stack,1)  # Roll channels to the front
    
    # Run pipeline
    output_dict = {}
    for c,n in zip(range(stack.shape[0]), c_names):
        lms, lms_tf, lum_dist_lms = pcl.point_cloud_pipeline(stack[c,...], stack[0,...],
                                                             fpath, fname, res, num_LMs=1000)
        output_dict['lm_memsub_'+n]       = lms
        output_dict['lm_memsub_'+n+'_tf'] = lms_tf
        output_dict['memsub_lum_dist_'+n] = lum_dist_lms
    
    # Save results
    np.savez(os.path.join(fpath, fname[:-4]+'_LMs'), **output_dict)

# report
print "### ALL DONE! ###"